In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib widget
import matplotlib.pyplot as plt
from scipy.constants import R
import pandas as pd
import numpy as np
import os
import glob 

import ipywidgets as widgets

import pygaps as pg
from bet_class import BET

In [ ]:
layout=widgets.Layout(width='50%', height='80px')


##### WIDGET DEFINITIONS



main_folder = widgets.Text(description='Main folder', layout=layout)  #  Main folder select
select_folder = widgets.Dropdown(description='Folder Select', layout=widgets.Layout(width='450px', height='30px'))
file_folder = widgets.SelectMultiple(description='File Select', layout=widgets.Layout(width='450px', height='350px'))

run_analysis = widgets.Button(description='Run BET', disabled=True)

clear_all_b = widgets.Button(description='Clear All')

###### BET PLOTTING WIDGETS

isotherm_out = widgets.Output()
bet_plot = widgets.Output()
roq_plot = widgets.Output()
bet_res = widgets.Output()

plot_outputs = [isotherm_out, bet_plot, roq_plot, bet_res]

#### WIDGET EVENTS

def on_value_change(change):
    folder = os.listdir(change.new)
    select_folder.options=folder

def on_value_change_2(change):
    fold = os.path.join(main_folder.value, change.new)
    files = [_ for _ in os.listdir(fold) if _.endswith('.DAT')]
    file_folder.options = files
    run_analysis.disabled = False
    
def run_bet(b):
    file_select = os.path.join(main_folder.value, select_folder.value, file_folder.value[0])
    bet = BET(file_select)
    with bet_res:
        res = bet.BET_results 
        df = pd.DataFrame(res.values(), columns=['value'])
        df = df.set_index(pd.Index([*res]))
        display(df)
        
    with isotherm_out:
        bet.plot_isotherm()
    
    with bet_plot:
        bet.plot_bet()

    with roq_plot:
        bet.plot_roq()
        
def clear_all(b):
    for output in plot_outputs:
        output.clear_output()
    
    
main_folder.observe(on_value_change, names='value')
select_folder.observe(on_value_change_2, names='value')

run_analysis.on_click(run_bet)
clear_all_b.on_click(clear_all)

##### WIDGET LAYOUT

a = widgets.VBox([select_folder, file_folder], layout=widgets.Layout(width='500px'))
BET_plots = widgets.HBox([bet_plot, roq_plot])

buttons = widgets.HBox([run_analysis, clear_all_b])

b = widgets.VBox([buttons, bet_res])
c = widgets.HBox(([b, BET_plots]))


d = widgets.HBox([a, c])




main = widgets.VBox([main_folder, d, isotherm_out])    
display(main)